This assignment covers non_linear transformation, validation bias, PLA vs. SVM, etc. 
Problem set available here for reference: http://work.caltech.edu/homework/hw7.pdf

In [1]:
import numpy as np
from sklearn.svm import SVC
from random import uniform
from random import choice

In [2]:
# Peforms (x1, x1) -> (1, x1, x2, x1^2, x2^2, x1x2, |x1 - x2|, |x1 + x2|)
def nonlin_transform(x, phi_i):
    n = x.shape[0]
    phi = np.ones((n,1))
    phi = np.hstack((phi, x))
  
    if phi_i >= 3:
        phi = np.hstack((phi, np.reshape(x[:,0]**2, (n,1))))
    if phi_i >= 4:
        phi = np.hstack((phi, np.reshape(x[:,1]**2, (n,1))))
    if phi_i >= 5:
        phi = np.hstack((phi, np.reshape(x[:,0]*x[:,1], (n,1))))
    if phi_i >= 6:
        phi = np.hstack((phi, np.reshape(np.abs(x[:,0]-x[:,1]), (n,1))))
    if phi_i >= 7:
        phi = np.hstack((phi, np.reshape(np.abs(x[:,0]+ x[:,1]), (n,1))))

    return phi

# Calculates linear reg weight vector for matrix X and target vector y
def weights(X, y):
    pseudo = np.dot(np.linalg.inv((np.dot(X.T, X))), X.T)
    return np.dot(pseudo, y)

# Calculates out of sample error 
def error(w, x, y):
    N = 0
    
    for i in range(x.shape[0]):
        N += max(0, np.sign(-np.dot(w.T, x[i])*y[i]))
    
    return N/float(x.shape[0])

In [3]:
# Question 1

data = np.loadtxt('in.dta.txt')
training_set = data[0:25]
valid_set = data[25:]

for k in [3,4,5,6,7]:
    phi_train = nonlin_transform(training_set[:, [0,1]], k)
    phi_w = weights(phi_train, training_set[:,2])
    
    phi_valid = nonlin_transform(valid_set[:,[0,1]], k)
    err_valid = error(phi_w, phi_valid, valid_set[:,2])

    print("Validation classification error for k = {}: {}".format(k, err_valid))

Validation classification error for k = 3: 0.3
Validation classification error for k = 4: 0.5
Validation classification error for k = 5: 0.2
Validation classification error for k = 6: 0.0
Validation classification error for k = 7: 0.1


In [4]:
# Question 2

test_set = np.loadtxt('out.dta.txt')

for k in [3,4,5,6,7]:
    phi_train = nonlin_transform(training_set[:, [0,1]], k)
    phi_w = weights(phi_train, training_set[:,2])
    
    phi_test = nonlin_transform(test_set[:,[0,1]], k)
    err_test = error(phi_w, phi_test, test_set[:,2])
    
    print("Test classification error for k = {}: {}".format(k, err_test))

Test classification error for k = 3: 0.42
Test classification error for k = 4: 0.416
Test classification error for k = 5: 0.188
Test classification error for k = 6: 0.084
Test classification error for k = 7: 0.072


In [5]:
# Question 3

valid_set = data[0:25]
training_set = data[25:]

for k in [3,4,5,6,7]:
    phi_train = nonlin_transform(training_set[:, [0,1]], k)
    phi_w = weights(phi_train, training_set[:,2])
    
    phi_valid = nonlin_transform(valid_set[:,[0,1]], k)
    err_valid = error(phi_w, phi_valid, valid_set[:,2])

    print("Validation classification error for k = {}: {}".format(k, err_valid))

Validation classification error for k = 3: 0.28
Validation classification error for k = 4: 0.36
Validation classification error for k = 5: 0.2
Validation classification error for k = 6: 0.08
Validation classification error for k = 7: 0.12


In [6]:
# Question 4

for k in [3,4,5,6,7]:
    phi_train = nonlin_transform(training_set[:, [0,1]], k)
    phi_w = weights(phi_train, training_set[:,2])
    
    phi_test = nonlin_transform(test_set[:,[0,1]], k)
    err_test = error(phi_w, phi_test, test_set[:,2])
    
    print("Test classification error for k = {}: {}".format(k, err_test))

Test classification error for k = 3: 0.396
Test classification error for k = 4: 0.388
Test classification error for k = 5: 0.284
Test classification error for k = 6: 0.192
Test classification error for k = 7: 0.196


In [7]:
# Question 6

e1 = [uniform(0, 1) for x in range(10**5)]
e2 = [uniform(0, 1) for x in range(10**5)]
e = [min(e1, e2) for e1, e2 in zip(e1, e2)]

print("e1 = {}, e2 = {}, e = {}".format(np.mean(e1), np.mean(e2), np.mean(e)))

e1 = 0.5006503512080165, e2 = 0.5008693739695061, e = 0.33439434859428746


In [8]:
# Generates the random line 
def gen_random_line():
    x1 = uniform(-1, 1)
    y1 = uniform(-1, 1)
    x2 = uniform(-1, 1)
    y2 = uniform(-1, 1)
    
    w = np.array([x2*y1-y2*x1, y2-y1, x1-x2])
    w_norm = np.array([1, -w[1]/w[2], -w[0]/w[2]])
    return w, w_norm


# Generates n random points and evaluates target function
def gen_random_points(n, w=None, w_norm=None):
    if w is None:
        w, w_norm = gen_random_line()
    
    y = [1]
    while len(set(y)) <= 1:
        d_ = np.random.uniform(-1.0, 1.0,(2,n))
        x_ = np.append(np.ones(n), d_).reshape((2+1,n))
        y = np.sign(np.dot(w.T,x_))
        d_ = np.append(x_, y).reshape((4,n))
    return x_, y, w, d_, w_norm

# Extracts the misclassified points 
def find_misclassified(y_, y):
    mis = []
    for i in range(len(y)):
        if y_[i] != y[i]:
            mis.append(i)
    
    try:
        index = choice(mis)
    except IndexError:
        index = 0
    
    return index, len(mis)

# Runs PLA algorithm using functions above
def pla(x_, y):
    w_ = np.zeros(3)
    y_ = np.sign(np.dot(w_.T,x_))

    while np.array_equal(y, y_) != True:
        index, total_mc_pts= find_misclassified(y_,y)
        w_ += y[index] * x_[:,index]
        y_ = np.sign(np.dot(w_.T, x_))

    w_n = np.array([1, -w_[1]/w_[2], -w_[0]/w_[2]])

    return i, w_n, w_

In [9]:
# Question 8

n = 10
pla_dis = []
svm_dis = []

for i in range(1000):
    x_, y, w, d_, w_n = gen_random_points(n)
    _, w_n_, w_ = pla(x_, y)             # pla
    clf = SVC(C=np.inf, kernel='linear') # svm
    clf.fit(x_[1:].T, y)
    
    x_, y, _, _, _ = gen_random_points(10000, w, w_n)  
    y_ = np.sign(np.dot(w_.T,x_))
    zzz, nmc = find_misclassified(y_, y)
    
    pla_dis.append(nmc)
    
    y_ = clf.predict(x_[1:].T) # svm
    zzz, nmc = find_misclassified(y_, y)
    
    svm_dis.append(nmc)

diff = np.array(svm_dis) - np.array(pla_dis)

print("improvement over PLA: {}%".format(sum(1 for i in diff if i < 0)/float(len(diff)) * 100))

improvement over PLA: 61.6%


In [10]:
# Question 9

n = 100
pla_dis = []
svm_dis = []
sv = []

for i in range(1000):
    x_, y, w, d_, w_n = gen_random_points(n)
    _, w_n_, w_ = pla(x_, y)             # pla
    clf = SVC(C=np.inf, kernel='linear') # svm
    clf.fit(x_[1:].T, y)
    
    x_, y, _, _, _ = gen_random_points(10000, w, w_n)  
    y_ = np.sign(np.dot(w_.T,x_))
    zzz, nmc = find_misclassified(y_, y)
    
    pla_dis.append(nmc)
    
    y_ = clf.predict(x_[1:].T) # svm
    zzz, nmc = find_misclassified(y_, y)
    
    svm_dis.append(nmc)
    
    sv.append(len(clf.support_vectors_))

diff = np.array(svm_dis) - np.array(pla_dis)

print("improvement over PLA: {}%".format(sum(1 for i in diff if i < 0)/float(len(diff)) * 100))

improvement over PLA: 65.10000000000001%


In [11]:
# Question 9

print("avg support vectors: {}".format(np.mean(sv)))

avg support vectors: 3.001
